In [ ]:
# plot total profit (over a year, fee included) vs trailing stop loss size.

In [ ]:
import pandas as pd
import numpy as np
import forecast
from tsl import get_trailing_stop_loss_long_profits
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fee_percent = 0.07

In [ ]:
data = forecast.get_df('BTCUSD', '', '1T')

In [ ]:
(profits, long_stop_loss_prices, maximums) = get_trailing_stop_loss_long_profits(data, 0.5 * 0.01)

In [ ]:
def get_total_profits(df):
    n = len(profits)
    woFee = (df['profit'] - df['open'] * 0.01*fee_percent*2).to_numpy()
    total_profit = 0
    for i in range(1, n):
        if woFee[i - 1] < 0 and woFee[i] > 0:
            total_profit += woFee[i]
    return total_profit

In [ ]:
stop_losses = []
total_profits = []
for sl in np.arange(0.05 * 0.01, 0.3 * 0.01, 0.02 * 0.01):
    (profits, stop_loss_prices, maximums) = get_trailing_stop_loss_long_profits(data, sl)
    profits = pd.DataFrame(profits)
    profits.ffill(inplace=True)
    data['profit'] = profits.to_numpy()
    total_profit = get_total_profits(data)
    stop_losses.append(sl)
    total_profits.append(total_profit)
    print(f'sl={sl}, total_profit={total_profit}')

In [ ]:
fig = make_subplots(rows=1, cols=1, shared_xaxes=True, 
            vertical_spacing=0.01,
            row_heights=[1],
            specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=stop_losses, y=total_profits, mode='lines', line=dict(color='black'),
                            name='total_profits'), row=1, col=1)

In [ ]:
(0.01 * 0.07 * 2, 0.01 * 0.05 * 2)

In [ ]:
""" i don't know if it's a coincidence or not but the optimal trailing stop loss size happens to be close to trading 
fees. for example, on bybit trading fee starts at 0.1%. and the optimal trailing stop loss is 0.11%.
"""